In [2]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

import os
import xarray as xr
import numpy as np
import pickle
from esio import EsioData as ed
import itertools
import seaborn as sns

In [3]:
# Update your paths here
my_data_dir = '/home/disk/sipn/nicway/data/' # Path to where SIPN2 data files are stored (where the pickle file gets stored)
public_html_fig = r'/home/disk/sipn/nicway/public_html/sipn/figures/' # Where figs for website go

In [4]:
# Create an esioData
E = ed.EsioData(data_dir = my_data_dir)

In [5]:
# Add observations
obs_list = ['G10006',  'NSIDC_0051',  'NSIDC_0079',  'NSIDC_0081']
for co in obs_list:
    E.add_obs(co, grid_file='stereo_gridinfo.nc')
E.add_obs('NSIDC_extent', grid_file='N/A')    

In [6]:
# Add Models

# Statistical models
#E.add_model(mod_name='dampedAnomaly', model_label='Damped\n persistence', grid_file='MISSING') # Removed because it was terrible
E.add_model(mod_name='dampedAnomalyTrend', model_label='Damped\nAnomaly', grid_file='MISSING')

# Dynamical models
E.add_model(mod_name='gfdlsipn', model_label='GFDL-FLOR', grid_file='GFDLFLOR_gridinfo.nc')
E.add_model(mod_name='piomas', model_label='PIOMAS', grid_file='PIOMAS_gridinfo.nc')
E.add_model(mod_name='yopp', model_label='ECMWF-YOPP*', grid_file='yopp_land_sea_mask.nc')
# c3S Models
E.add_model(mod_name='ukmetofficesipn', model_label='UKMO-C3S*', grid_file='ukmetofficesipn_land_sea_mask.nc')
E.add_model(mod_name='ecmwfsipn', model_label='ECMWF-C3S*', grid_file='ecmwfsipn_land_sea_mask.nc')
# S2S download models
E.add_model(mod_name='bom', model_label='BOM-S2S', grid_file='MISSING')
E.add_model(mod_name='cma', model_label='CMA-S2S', grid_file='MISSING')
E.add_model(mod_name='ecmwf', model_label='ECMWF-S2S', grid_file='MISSING')
E.add_model(mod_name='hcmr', model_label='HCMR-S2S', grid_file='MISSING')
E.add_model(mod_name='isaccnr', model_label='ISAC-CNR-S2S', grid_file='MISSING')
E.add_model(mod_name='jma', model_label='JMA-S2S', grid_file='MISSING')
E.add_model(mod_name='metreofr', model_label='MÉTÉO-FR-S2S', grid_file='MISSING')
E.add_model(mod_name='ukmo', model_label='UKMO-S2S', grid_file='MISSING')
E.add_model(mod_name='eccc', model_label='ECCC-S2S', grid_file='MISSING')
E.add_model(mod_name='kma', model_label='KMA-S2S', grid_file='MISSING')
E.add_model(mod_name='ncep', model_label='NCEP-S2S', grid_file='MISSING')
# NAVY NEMO
E.add_model(mod_name='usnavysipn', model_label='NRL NESM EXT', grid_file='MISSING')
E.add_model(mod_name='usnavyncep', model_label='NRL NESM', grid_file='MISSING')
# RASM ESRL
E.add_model(mod_name='rasmesrl', model_label='CAFS', grid_file='MISSING')
# NECP CFSv2
E.add_model(mod_name='noaasipn', model_label='NCEP-EXP', grid_file='MISSING')
E.add_model(mod_name='noaasipn_ext', model_label='NCEP-EXT', grid_file='MISSING')
E.add_model(mod_name='usnavygofs', model_label='NRL-GOFS', grid_file='MISSING')

# E.add_model(mod_name='noaasipn_ext_biascorr', model_label='ncep-ext-biascorr', grid_file='MISSING')

E.add_model(mod_name='modcansipns_3', model_label='CANSIPS-3', grid_file='MISSING')
E.add_model(mod_name='modcansipns_4', model_label='CANSIPS-4', grid_file='MISSING')
E.add_model(mod_name='uclsipn', model_label='UCL', grid_file='mesh_mask_nemo.N3.6_ORCA1L75.nc')

E.add_model(mod_name='szapirosipn', model_label='MPAS-CESM', grid_file='MISSING')

E.add_model(mod_name='awispin', model_label='AWI', grid_file='MISSING') #\n(Kauker et al.)
E.add_model(mod_name='nicosipn', model_label='NICO SUN', grid_file='MISSING')
E.add_model(mod_name='fgoalssipn', model_label='IAP LASG', grid_file='MISSING')


# MME
E.add_model(mod_name='MME', model_label='MME Mean', grid_file='MISSING')
E.add_model(mod_name='MME_NEW', model_label='MME Mean', grid_file='MISSING')


In [7]:
# Add other path info
E.fig_dir = public_html_fig 

In [8]:
# Flag if modeled ice is predicted (True) or prescribed (False)
icePredicted = {'gfdlsipn':True, 'piomas':True, 'yopp':True, 'bom':False, 'cma':False, 'ecmwf':True, 
              'hcmr':False, 'isaccnr':False, 'jma':False, 'metreofr':True, 'ukmo':True, 'eccc':False, 
              'kma':True, 'ncep':True, 'ukmetofficesipn':True, 'ecmwfsipn':True, 'usnavysipn':True,
               'usnavyncep':True, 'rasmesrl':True, 'noaasipn':True, 'noaasipn_ext':True,
               'MME':True, 'modcansipns_3':True, 'modcansipns_4':True, 'uclsipn':True, 'szapirosipn':True,
               'awispin':True, 'nicosipn':True, 'usnavygofs':True, 'MME_NEW':True,
               'dampedAnomalyTrend':True, 'fgoalssipn':True}
E.icePredicted = icePredicted

In [9]:
# Check all models have a predicted flag
if not (icePredicted.keys() == E.model.keys()):
    raise ValueError("Missing some predicted flags for models...")

In [10]:
E.model.keys()

dict_keys(['dampedAnomalyTrend', 'gfdlsipn', 'piomas', 'yopp', 'ukmetofficesipn', 'ecmwfsipn', 'bom', 'cma', 'ecmwf', 'hcmr', 'isaccnr', 'jma', 'metreofr', 'ukmo', 'eccc', 'kma', 'ncep', 'usnavysipn', 'usnavyncep', 'rasmesrl', 'noaasipn', 'noaasipn_ext', 'usnavygofs', 'modcansipns_3', 'modcansipns_4', 'uclsipn', 'szapirosipn', 'awispin', 'nicosipn', 'fgoalssipn', 'MME', 'MME_NEW'])

In [11]:
icePredicted.keys()

dict_keys(['gfdlsipn', 'piomas', 'yopp', 'bom', 'cma', 'ecmwf', 'hcmr', 'isaccnr', 'jma', 'metreofr', 'ukmo', 'eccc', 'kma', 'ncep', 'ukmetofficesipn', 'ecmwfsipn', 'usnavysipn', 'usnavyncep', 'rasmesrl', 'noaasipn', 'noaasipn_ext', 'MME', 'modcansipns_3', 'modcansipns_4', 'uclsipn', 'szapirosipn', 'awispin', 'nicosipn', 'usnavygofs', 'MME_NEW', 'dampedAnomalyTrend', 'fgoalssipn'])

In [12]:
E.model['ecmwf']['model_label']

'ECMWF-S2S'

In [13]:
list(zip([E.model[x]['model_label'] for x in E.model.keys()], E.model.keys()))

[('Damped\nAnomaly', 'dampedAnomalyTrend'),
 ('GFDL-FLOR', 'gfdlsipn'),
 ('PIOMAS', 'piomas'),
 ('ECMWF-YOPP*', 'yopp'),
 ('UKMO-C3S*', 'ukmetofficesipn'),
 ('ECMWF-C3S*', 'ecmwfsipn'),
 ('BOM-S2S', 'bom'),
 ('CMA-S2S', 'cma'),
 ('ECMWF-S2S', 'ecmwf'),
 ('HCMR-S2S', 'hcmr'),
 ('ISAC-CNR-S2S', 'isaccnr'),
 ('JMA-S2S', 'jma'),
 ('MÉTÉO-FR-S2S', 'metreofr'),
 ('UKMO-S2S', 'ukmo'),
 ('ECCC-S2S', 'eccc'),
 ('KMA-S2S', 'kma'),
 ('NCEP-S2S', 'ncep'),
 ('NRL NESM EXT', 'usnavysipn'),
 ('NRL NESM', 'usnavyncep'),
 ('CAFS', 'rasmesrl'),
 ('NCEP-EXP', 'noaasipn'),
 ('NCEP-EXT', 'noaasipn_ext'),
 ('NRL-GOFS', 'usnavygofs'),
 ('CANSIPS-3', 'modcansipns_3'),
 ('CANSIPS-4', 'modcansipns_4'),
 ('UCL', 'uclsipn'),
 ('MPAS-CESM', 'szapirosipn'),
 ('AWI', 'awispin'),
 ('NICO SUN', 'nicosipn'),
 ('IAP LASG', 'fgoalssipn'),
 ('MME Mean', 'MME'),
 ('MME Mean', 'MME_NEW')]

In [14]:
# Define colors/lines etce for each model to use in plots
cmap_c = itertools.cycle(sns.color_palette("Paired", len(E.model.keys()) ))
linecycler = itertools.cycle(["-","--","-.",":","--"])
markercycler = itertools.cycle(["*","o","s","v","X"])
mod_cc = {}
mod_ls = {}
mod_mk = {}
for (i, cmod) in enumerate(E.model.keys()):
    mod_cc[cmod] = next(cmap_c)
    mod_ls[cmod] = next(linecycler)
    mod_mk[cmod] = next(markercycler)
E.model_color = mod_cc
E.model_linestyle = mod_ls
E.model_marker= mod_mk

# CC adjusted to her liking
E.model_color['usnavygofs']='tab:olive'
E.model_color['usnavyncep']='tab:olive'
E.model_color['ncep']='blue'
E.model_linestyle['usnavyncep']='-'
E.model_linestyle['ncep']='-'
E.model_linestyle['ecmwf']='-'

In [15]:
# Save to ESIO dir
print("Saving to ", my_data_dir)
E.save()

Saving to  /home/disk/sipn/nicway/data/


In [16]:
# # Load object to test
# E = None
# E = ed.EsioData.load()

In [17]:
import sys
print(sys.path)
print(sys.prefix)

['', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python36.zip', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/lib-dynload', '/home/disk/eos11/bitz/.local/lib/python3.6/site-packages', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/PyNIO', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/cycler-0.10.0-py3.6.egg', '/home/disk/sipn/nicway/python/ESIO', '/home/disk/sipn/nicway/python/xskillscore', '/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/IPython/extensions', '/home/disk/eos11/bitz/.ipython']
/home/disk/sipn/nicway/anaconda3/envs/esio
